<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Cross_Tab_Cube_Rollup_Grouping_Sets_PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

https://www.sqltutorial.org/sql-grouping-sets/ <br>
https://www.sqltutorial.org/sql-rollup/ <br>
https://www.sqltutorial.org/sql-cube/

#Install

In [1]:
# Install postgresql server
!apt update > /dev/null
!apt install postgresql > /dev/null
!service postgresql start
!sudo -u postgres psql -V





 * Starting PostgreSQL 12 database server
   ...done.
psql (PostgreSQL) 12.14 (Ubuntu 12.14-0ubuntu0.20.04.1)


## Configure

In [2]:
# Setup a password `pass` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Setup a database with name `praxisdb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [3]:
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:praxisdb:postgres:pass


## Create Table

In [4]:
#!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c \
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c \
"CREATE TABLE inventory ( \
    warehouse VARCHAR(255), \
    product VARCHAR(255) NOT NULL, \
    model VARCHAR(50) NOT NULL, \
    quantity INT, \
    PRIMARY KEY (warehouse,product,model) \
); \
"

CREATE TABLE


## Load Data

In [5]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Jose', 'iPhone','6s',100); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Fransisco', 'iPhone','6s',50); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Jose','iPhone','7',50); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Fransisco', 'iPhone','7',10); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Jose','iPhone','X',150); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Fransisco', 'iPhone','X',200); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Jose','Samsung','Galaxy S',200); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Fransisco','Samsung','Galaxy S',200); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Fransisco','Samsung','Note 8',100); \
INSERT INTO inventory(warehouse, product, model, quantity) VALUES('San Jose','Samsung','Note 8',150); \
"

INSERT 0 1


## View Data

In [6]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from inventory"

   warehouse   | product |  model   | quantity 
---------------+---------+----------+----------
 San Jose      | iPhone  | 6s       |      100
 San Fransisco | iPhone  | 6s       |       50
 San Jose      | iPhone  | 7        |       50
 San Fransisco | iPhone  | 7        |       10
 San Jose      | iPhone  | X        |      150
 San Fransisco | iPhone  | X        |      200
 San Jose      | Samsung | Galaxy S |      200
 San Fransisco | Samsung | Galaxy S |      200
 San Fransisco | Samsung | Note 8   |      100
 San Jose      | Samsung | Note 8   |      150
(10 rows)



#Grouping Sets

In [7]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM (quantity) qty \
FROM inventory \
GROUP BY warehouse, product; \
"

   warehouse   | product | qty 
---------------+---------+-----
 San Fransisco | iPhone  | 260
 San Fransisco | Samsung | 300
 San Jose      | iPhone  | 300
 San Jose      | Samsung | 350
(4 rows)



In [8]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, SUM (quantity) qty \
FROM inventory \
GROUP BY warehouse; \
"

   warehouse   | qty 
---------------+-----
 San Fransisco | 560
 San Jose      | 650
(2 rows)



In [9]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT product, SUM (quantity) qty \
FROM inventory \
GROUP BY product; \
"

 product | qty 
---------+-----
 iPhone  | 560
 Samsung | 650
(2 rows)



In [10]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT SUM(quantity) qty \
FROM inventory; \
"

 qty  
------
 1210
(1 row)



## Union All

In [11]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM (quantity) qty \
FROM inventory GROUP BY warehouse, product \
UNION ALL\
SELECT  warehouse, null, SUM (quantity) qty \
FROM inventory GROUP BY warehouse \
UNION ALL \
SELECT null, product, SUM (quantity) qty \
FROM inventory GROUP BY product \
UNION ALL\
SELECT null, null, SUM(quantity) qty \
FROM inventory;\
"

   warehouse   | product | qty  
---------------+---------+------
 San Fransisco | iPhone  |  260
 San Fransisco | Samsung |  300
 San Jose      | iPhone  |  300
 San Jose      | Samsung |  350
 San Fransisco |         |  560
 San Jose      |         |  650
               | iPhone  |  560
               | Samsung |  650
               |         | 1210
(9 rows)



## Grouping Sets

In [12]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product,  SUM (quantity) qty \
FROM inventory \
GROUP BY \
    GROUPING SETS( \
        (warehouse,product), \
        (warehouse), \
        (product), \
        () \
    ); \
"


   warehouse   | product | qty  
---------------+---------+------
               |         | 1210
 San Fransisco | iPhone  |  260
 San Fransisco | Samsung |  300
 San Jose      | iPhone  |  300
 San Jose      | Samsung |  350
 San Fransisco |         |  560
 San Jose      |         |  650
               | iPhone  |  560
               | Samsung |  650
(9 rows)



#Roll Up

In [13]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, SUM(quantity) \
FROM inventory \
GROUP BY warehouse; \
"

   warehouse   | sum 
---------------+-----
 San Fransisco | 560
 San Jose      | 650
(2 rows)



In [14]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, SUM(quantity) \
FROM inventory \
GROUP BY ROLLUP (warehouse); \
"

   warehouse   | sum  
---------------+------
               | 1210
 San Fransisco |  560
 San Jose      |  650
(3 rows)



In [15]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(warehouse, 'All warehouses') AS warehouse, \
    SUM(quantity) \
FROM inventory \
GROUP BY ROLLUP (warehouse);\
"

   warehouse    | sum  
----------------+------
 All warehouses | 1210
 San Fransisco  |  560
 San Jose       |  650
(3 rows)



## Multi Columns

In [16]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM(quantity) \
FROM inventory \
GROUP BY warehouse, product; \
"

   warehouse   | product | sum 
---------------+---------+-----
 San Fransisco | iPhone  | 260
 San Fransisco | Samsung | 300
 San Jose      | iPhone  | 300
 San Jose      | Samsung | 350
(4 rows)



In [17]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM(quantity) \
FROM inventory \
GROUP BY ROLLUP (warehouse , product); \
"

   warehouse   | product | sum  
---------------+---------+------
               |         | 1210
 San Fransisco | iPhone  |  260
 San Fransisco | Samsung |  300
 San Jose      | iPhone  |  300
 San Jose      | Samsung |  350
 San Fransisco |         |  560
 San Jose      |         |  650
(7 rows)



## Partial Roll Up

In [18]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM(quantity) \
FROM inventory \
GROUP BY warehouse, ROLLUP (product); \
"

   warehouse   | product | sum 
---------------+---------+-----
 San Fransisco | iPhone  | 260
 San Fransisco | Samsung | 300
 San Jose      | iPhone  | 300
 San Jose      | Samsung | 350
 San Fransisco |         | 560
 San Jose      |         | 650
(6 rows)



In [19]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, COALESCE(product, 'All products'), SUM(quantity) \
FROM inventory \
GROUP BY warehouse, ROLLUP (product); \
"

   warehouse   |   coalesce   | sum 
---------------+--------------+-----
 San Fransisco | iPhone       | 260
 San Fransisco | Samsung      | 300
 San Jose      | iPhone       | 300
 San Jose      | Samsung      | 350
 San Fransisco | All products | 560
 San Jose      | All products | 650
(6 rows)



#Cube

In [20]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, SUM(quantity) \
FROM inventory \
GROUP BY warehouse; \
"

   warehouse   | sum 
---------------+-----
 San Fransisco | 560
 San Jose      | 650
(2 rows)



In [21]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, SUM(quantity) \
FROM inventory \
GROUP BY CUBE(warehouse) \
ORDER BY warehouse;  \
"

   warehouse   | sum  
---------------+------
 San Fransisco |  560
 San Jose      |  650
               | 1210
(3 rows)



In [22]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(warehouse, 'All warehouses'), SUM(quantity) \
FROM inventory \
GROUP BY CUBE(warehouse) \
ORDER BY warehouse;  \
"

    coalesce    | sum  
----------------+------
 San Fransisco  |  560
 San Jose       |  650
 All warehouses | 1210
(3 rows)



## Multiple Columns

In [24]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT warehouse, product, SUM(quantity) \
FROM inventory \
GROUP BY warehouse,product \
ORDER BY warehouse,product; \
"

   warehouse   | product | sum 
---------------+---------+-----
 San Fransisco | iPhone  | 260
 San Fransisco | Samsung | 300
 San Jose      | iPhone  | 300
 San Jose      | Samsung | 350
(4 rows)



In [25]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT \
   warehouse, \
   product, \
   SUM(quantity) \
FROM  inventory \
GROUP BY CUBE(warehouse,product) \
ORDER BY warehouse, product; \
"


   warehouse   | product | sum  
---------------+---------+------
 San Fransisco | iPhone  |  260
 San Fransisco | Samsung |  300
 San Fransisco |         |  560
 San Jose      | iPhone  |  300
 San Jose      | Samsung |  350
 San Jose      |         |  650
               | iPhone  |  560
               | Samsung |  650
               |         | 1210
(9 rows)



In [26]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT \
   COALESCE(warehouse, '...All Warehouses') warehouse, \
   COALESCE(product, '...All Products') product, \
   SUM(quantity) \
FROM  inventory \
GROUP BY CUBE(warehouse,product) \
ORDER BY warehouse, product; \
"

     warehouse     |     product     | sum  
-------------------+-----------------+------
 ...All Warehouses | ...All Products | 1210
 ...All Warehouses | iPhone          |  560
 ...All Warehouses | Samsung         |  650
 San Fransisco     | ...All Products |  560
 San Fransisco     | iPhone          |  260
 San Fransisco     | Samsung         |  300
 San Jose          | ...All Products |  650
 San Jose          | iPhone          |  300
 San Jose          | Samsung         |  350
(9 rows)



#Three Dimensional Cube
see [MySQL Version](https://github.com/Praxis-QR/RDWH/blob/main/Cross_Tabs_with_OLAP_Cube_Normalised_KeyValueTables.ipynb) for context

## Create Fact Table

In [27]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c 'DROP TABLE IF EXISTS CarSalesFact_S2'

NOTICE:  table "carsalesfact_s2" does not exist, skipping
DROP TABLE


In [28]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
CREATE TABLE CarSalesFact_S2 ( \
  Make text, \
  Type text, \
  Colour text, \
  Value int DEFAULT NULL,\
  Units int NOT NULL DEFAULT '0'\
)"

CREATE TABLE


In [29]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
INSERT INTO CarSalesFact_S2 VALUES \
('Honda','SUV','Red',7000,2),('Honda','SUV','Black',18000,4),('Honda','Sedan','Brown',5000,1), \
('Toyota','Compact','Red',7000,1),('Toyota','Sedan','Black',9000,2),\
('Mazda','SUV','Black',4000,1),('Mazda','Compact','Red',5000,1),('Mazda','Sedan','Brown',4000,1),('Mazda','SUV','Red',3000,1);\
"

INSERT 0 9


In [30]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from CarSalesFact_S2"  

  make  |  type   | colour | value | units 
--------+---------+--------+-------+-------
 Honda  | SUV     | Red    |  7000 |     2
 Honda  | SUV     | Black  | 18000 |     4
 Honda  | Sedan   | Brown  |  5000 |     1
 Toyota | Compact | Red    |  7000 |     1
 Toyota | Sedan   | Black  |  9000 |     2
 Mazda  | SUV     | Black  |  4000 |     1
 Mazda  | Compact | Red    |  5000 |     1
 Mazda  | Sedan   | Brown  |  4000 |     1
 Mazda  | SUV     | Red    |  3000 |     1
(9 rows)



## Cross Tab Level 1

In [31]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact_S2 \
  group by Make, Type \
"

  make  |  type   | count 
--------+---------+-------
 Mazda  | SUV     |     2
 Honda  | SUV     |     6
 Mazda  | Sedan   |     1
 Toyota | Compact |     1
 Toyota | Sedan   |     2
 Honda  | Sedan   |     1
 Mazda  | Compact |     1
(7 rows)



In [32]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
  select Make,  Sum(Units) Count \
  from CarSalesFact_S2 \
  group by Make \
"

  make  | count 
--------+-------
 Honda  |     7
 Toyota |     3
 Mazda  |     4
(3 rows)



In [33]:
#case when (Dim = 'Make') then DimValue else '' end
#case when (Type = 'Sedan') then Units else 0 end


!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
    select Make, \
    sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan, \
    sum(case when (Type = 'SUV') then Units else 0 end) as SUV, \
    sum(case when (Type = 'Compact') then Units else 0 end) as Compact \
    from CarSalesFact_S2 \
    group by Make"

  make  | sedan | suv | compact 
--------+-------+-----+---------
 Honda  |     1 |   6 |       0
 Toyota |     2 |   0 |       1
 Mazda  |     1 |   2 |       1
(3 rows)



In [35]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
    select Make, \
    sum(case when (Type = 'Sedan') then Value else 0 end) as Sedan, \
    sum(case when (Type = 'SUV') then Value else 0 end) as SUV, \
    sum(case when (Type = 'Compact') then Value else 0 end) as Compact \
    from CarSalesFact_S2 \
    group by Make"

  make  | sedan |  suv  | compact 
--------+-------+-------+---------
 Honda  |  5000 | 25000 |       0
 Toyota |  9000 |     0 |    7000
 Mazda  |  4000 |  7000 |    5000
(3 rows)



## Cross Tab Level 2

In [37]:
#!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
#  select Make, Type, Sum(Units) Count \
#  from CarSalesFact_S2 \
#  group by Make, Type  \
#  with ROLLUP; \
#"

#COALESCE(warehouse, 'All warehouses') AS warehouse,

!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
  select COALESCE(Make, 'All Makes') AS Make, \
  COALESCE(Type, 'All Types') AS Type, \
  Sum(Units) Count \
  from CarSalesFact_S2 \
  group by ROLLUP (Make, Type)  \
"

   make    |   type    | count 
-----------+-----------+-------
 All Makes | All Types |    14
 Mazda     | SUV       |     2
 Honda     | SUV       |     6
 Mazda     | Sedan     |     1
 Toyota    | Compact   |     1
 Toyota    | Sedan     |     2
 Honda     | Sedan     |     1
 Mazda     | Compact   |     1
 Honda     | All Types |     7
 Toyota    | All Types |     3
 Mazda     | All Types |     4
(11 rows)



## Cross Tab Level 3

In [38]:
#sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan, \
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
	Make,\
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S2\
  GROUP BY ROLLUP (Make)\
"

  make  | sedan | suv | compact 
--------+-------+-----+---------
        |     4 |   8 |       2
 Honda  |     1 |   6 |       0
 Toyota |     2 |   0 |       1
 Mazda  |     1 |   2 |       1
(4 rows)



In [39]:
#sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan, \
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
    COALESCE(Make, 'Total') AS Dim, \
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S2\
  GROUP BY ROLLUP (Make)\
"

  dim   | sedan | suv | compact 
--------+-------+-----+---------
 Total  |     4 |   8 |       2
 Honda  |     1 |   6 |       0
 Toyota |     2 |   0 |       1
 Mazda  |     1 |   2 |       1
(4 rows)



In [43]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
COALESCE(Make, 'Total') AS Dim, \
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
    Make, \
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S2\
  GROUP BY ROLLUP (Make)\
) AS sums \
"

  dim   | suv | sedan | compact | totals 
--------+-----+-------+---------+--------
 Total  |   8 |     4 |       2 |     14
 Honda  |   6 |     1 |       0 |      7
 Toyota |   0 |     2 |       1 |      3
 Mazda  |   2 |     1 |       1 |      4
(4 rows)



In [44]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
COALESCE(Make, 'Totals') AS Dim, \
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM ( \
  SELECT \
    Make, \
	sum(case when (Colour = 'Brown') then Units else 0 end) as Brown,\
	sum(case when (Colour = 'Black') then Units else 0 end) as Black,\
	sum(case when (Colour = 'Red') then Units else 0 end) as Red\
  FROM CarSalesFact_S2\
  GROUP BY ROLLUP (Make)\
) AS sums \
"

  dim   | brown | black | red | totals 
--------+-------+-------+-----+--------
 Totals |     2 |     7 |   5 |     14
 Honda  |     1 |     4 |   2 |      7
 Toyota |     0 |     2 |   1 |      3
 Mazda  |     1 |     1 |   2 |      4
(4 rows)



In [45]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from CarSalesFact_S2"  

  make  |  type   | colour | value | units 
--------+---------+--------+-------+-------
 Honda  | SUV     | Red    |  7000 |     2
 Honda  | SUV     | Black  | 18000 |     4
 Honda  | Sedan   | Brown  |  5000 |     1
 Toyota | Compact | Red    |  7000 |     1
 Toyota | Sedan   | Black  |  9000 |     2
 Mazda  | SUV     | Black  |  4000 |     1
 Mazda  | Compact | Red    |  5000 |     1
 Mazda  | Sedan   | Brown  |  4000 |     1
 Mazda  | SUV     | Red    |  3000 |     1
(9 rows)



In [46]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT \
COALESCE(make, '- All Makes -') make, \
COALESCE(Type, '- All types -') cType, \
COALESCE(colour, '- All colour -') colour, \
sum(Units) \
FROM CarSalesFact_S2 \
GROUP BY CUBE(make,Type,colour) \
ORDER BY make,Type,colour; \
"

     make      |     ctype     |     colour     | sum 
---------------+---------------+----------------+-----
 - All Makes - | Compact       | - All colour - |   2
 - All Makes - | Compact       | Red            |   2
 - All Makes - | Sedan         | - All colour - |   4
 - All Makes - | Sedan         | Black          |   2
 - All Makes - | Sedan         | Brown          |   2
 - All Makes - | SUV           | - All colour - |   8
 - All Makes - | SUV           | Black          |   5
 - All Makes - | SUV           | Red            |   3
 - All Makes - | - All types - | - All colour - |  14
 - All Makes - | - All types - | Black          |   7
 - All Makes - | - All types - | Brown          |   2
 - All Makes - | - All types - | Red            |   5
 Honda         | Sedan         | - All colour - |   1
 Honda         | Sedan         | Brown          |   1
 Honda         | SUV           | - All colour - |   6
 Honda         | SUV           | Black          |   4
 Honda         | SUV      

Signoff

In [47]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Singapore')))

Last tested   2023-03-15 16:59:04.431011+08:00


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

## Smaller Table, built with pivots - ignore

In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c 'DROP table if exists CarSalesFact_N'
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c 'CREATE TABLE if not exists CarSalesFact_N ( Txn int ,  Dim  varchar(8) ,  DimValue varchar(8) ) '

NOTICE:  table "carsalesfact_n" does not exist, skipping
DROP TABLE
CREATE TABLE


In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
INSERT INTO CarSalesFact_N (Txn, Dim, DimValue) VALUES \
 (1, 'Make', 'Honda'),(1, 'Type', 'SUV'),(1, 'Colour', 'Red'),\
 (2, 'Make', 'Honda'),(2, 'Type', 'SUV'),(2, 'Colour', 'Black'), \
 (3, 'Make', 'Honda'),(3, 'Type', 'Sedan'),(3, 'Colour', 'Brown'),\
 (4, 'Make', 'Toyota'),(4, 'Type', 'Compact'),(4, 'Colour', 'Red'),\
 (5, 'Make', 'Toyota'),(5, 'Colour', 'Black'),(5, 'Type', 'Sedan'),\
 (6, 'Make', 'Toyota'),(6, 'Colour', 'Black'),(6, 'Type', 'Sedan'),\
 (7, 'Make', 'Honda'),(7, 'Type', 'SUV'),(7, 'Colour', 'Black'),\
 (8, 'Make', 'Honda'),(8, 'Type', 'SUV'),(8, 'Colour', 'Red'),\
 (9, 'Make', 'Honda'),(9, 'Type', 'SUV'),(9, 'Colour', 'Black'),\
 (10, 'Make', 'Honda'),(10, 'Type', 'SUV'),(10, 'Colour', 'Black'),\
 (11, 'Make', 'Mazda'),(11, 'Type', 'SUV'),(11, 'Colour', 'Black'),\
 (12, 'Make', 'Mazda'),(12, 'Type', 'Compact'),(12, 'Colour', 'Red'),\
 (13, 'Make', 'Mazda'),(13, 'Type', 'Sedan'),(13, 'Colour', 'Brown'),\
 (14, 'Make', 'Mazda'),(14, 'Type', 'SUV'),(14, 'Colour', 'Red');\
 "

INSERT 0 42


In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
  Txn,\
  GROUP_CONCAT(if(Dim = 'Make', DimValue, NULL)) AS Make,\
  GROUP_CONCAT(if(Dim = 'Type', DimValue, NULL)) AS Type,\
  GROUP_CONCAT(if(Dim = 'Colour', DimValue, NULL)) AS Colour \
FROM CarSalesFact_N \
GROUP BY txn \
"

ERROR:  function if(boolean, character varying, unknown) does not exist
LINE 1:   SELECT    Txn,   GROUP_CONCAT(if(Dim = 'Make', DimValue, N...
                                        ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [ ]:
case when ((albums.id=allalbums.albumid) and (allalbums.photoid=$photoid)) then 1
     else 0
   end
case when (Dim = 'Make') then DimValue else NULL end

In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
  Txn,\
  array_agg(case when (Dim = 'Make') then DimValue else '' end) AS Make,\
  array_agg(case when (Dim = 'Type') then DimValue else '' end) AS Type,\
  array_agg(case when (Dim = 'Colour') then DimValue else '' end) AS Colour \
FROM CarSalesFact_N \
GROUP BY txn \
"

 txn |      make      |      type       |    colour     
-----+----------------+-----------------+---------------
   1 | {Honda,"",""}  | {"",SUV,""}     | {"","",Red}
   2 | {Honda,"",""}  | {"",SUV,""}     | {"","",Black}
   3 | {Honda,"",""}  | {"",Sedan,""}   | {"","",Brown}
   4 | {Toyota,"",""} | {"",Compact,""} | {"","",Red}
   5 | {Toyota,"",""} | {"","",Sedan}   | {"",Black,""}
   6 | {Toyota,"",""} | {"","",Sedan}   | {"",Black,""}
   7 | {Honda,"",""}  | {"",SUV,""}     | {"","",Black}
   8 | {Honda,"",""}  | {"",SUV,""}     | {"","",Red}
   9 | {Honda,"",""}  | {"",SUV,""}     | {"","",Black}
  10 | {Honda,"",""}  | {"",SUV,""}     | {"","",Black}
  11 | {Mazda,"",""}  | {"",SUV,""}     | {"","",Black}
  12 | {Mazda,"",""}  | {"",Compact,""} | {"","",Red}
  13 | {Mazda,"",""}  | {"",Sedan,""}   | {"","",Brown}
  14 | {Mazda,"",""}  | {"",SUV,""}     | {"","",Red}
(14 rows)



In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
  Txn,\
  array_to_string(array_agg(case when (Dim = 'Make') then DimValue else '' end),'') AS Make,\
  array_to_string(array_agg(case when (Dim = 'Type') then DimValue else '' end),'') AS Type,\
  array_to_string(array_agg(case when (Dim = 'Colour') then DimValue else '' end),'') AS Colour \
FROM CarSalesFact_N \
GROUP BY txn \
"

 txn |  make  |  type   | colour 
-----+--------+---------+--------
   1 | Honda  | SUV     | Red
   2 | Honda  | SUV     | Black
   3 | Honda  | Sedan   | Brown
   4 | Toyota | Compact | Red
   5 | Toyota | Sedan   | Black
   6 | Toyota | Sedan   | Black
   7 | Honda  | SUV     | Black
   8 | Honda  | SUV     | Red
   9 | Honda  | SUV     | Black
  10 | Honda  | SUV     | Black
  11 | Mazda  | SUV     | Black
  12 | Mazda  | Compact | Red
  13 | Mazda  | Sedan   | Brown
  14 | Mazda  | SUV     | Red
(14 rows)



In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
(SELECT \
  Txn,\
  array_to_string(array_agg(case when (Dim = 'Make') then DimValue else '' end),'') AS Make,\
  array_to_string(array_agg(case when (Dim = 'Type') then DimValue else '' end),'') AS Type,\
  array_to_string(array_agg(case when (Dim = 'Colour') then DimValue else '' end),'') AS Colour \
FROM CarSalesFact_N \
GROUP BY txn \
) as csf0 \
group by \
csf0.Make, csf0.Type, csf0.Colour \
"

  make  |  type   | colour | units 
--------+---------+--------+-------
 Honda  | SUV     | Red    |     2
 Honda  | SUV     | Black  |     4
 Toyota | Compact | Red    |     1
 Mazda  | Compact | Red    |     1
 Mazda  | SUV     | Red    |     1
 Toyota | Sedan   | Black  |     2
 Honda  | Sedan   | Brown  |     1
 Mazda  | Sedan   | Brown  |     1
 Mazda  | SUV     | Black  |     1
(9 rows)



In [ ]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "drop table if exists CarSalesFact_S"  
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
create table CarSalesFact_S as ( \
    select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
    (SELECT \
    Txn,\
    array_to_string(array_agg(case when (Dim = 'Make') then DimValue else '' end),'') AS Make,\
    array_to_string(array_agg(case when (Dim = 'Type') then DimValue else '' end),'') AS Type,\
    array_to_string(array_agg(case when (Dim = 'Colour') then DimValue else '' end),'') AS Colour \
    FROM CarSalesFact_N \
    GROUP BY txn \
    ) as csf0 \
    group by \
    csf0.Make, csf0.Type, csf0.Colour \
) \
"
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from CarSalesFact_S"  

DROP TABLE
SELECT 9
  make  |  type   | colour | units 
--------+---------+--------+-------
 Honda  | SUV     | Red    |     2
 Honda  | SUV     | Black  |     4
 Toyota | Compact | Red    |     1
 Mazda  | Compact | Red    |     1
 Mazda  | SUV     | Red    |     1
 Toyota | Sedan   | Black  |     2
 Honda  | Sedan   | Brown  |     1
 Mazda  | Sedan   | Brown  |     1
 Mazda  | SUV     | Black  |     1
(9 rows)

